<a href="https://colab.research.google.com/github/bitblayde/Machine-and-Deep-learning-projects/blob/main/NLP/generate_shakespearean_text/statelessRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import numpy as np
import os
import sklearn
import tensorflow as tf
from tensorflow import keras

shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"

file = keras.utils.get_file("shakespeare.txt", shakespeare_url)

with open(file) as f:
  text = f.read()

In [52]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(text)

In [53]:
tokenizer.texts_to_sequences(["Hello"])

[[7, 2, 12, 12, 4]]

In [54]:
''.join(tokenizer.sequences_to_texts([[7, 2, 12, 12, 4]]))

'h e l l o'

In [55]:
n_characters = len(tokenizer.word_counts)
n_characters, tokenizer.word_counts

(39,
 OrderedDict([('f', 17567),
              ('i', 57369),
              ('r', 53758),
              ('s', 54219),
              ('t', 74024),
              (' ', 169892),
              ('c', 19443),
              ('z', 554),
              ('e', 100652),
              ('n', 53608),
              (':', 10316),
              ('\n', 40000),
              ('b', 14082),
              ('o', 71279),
              ('w', 21115),
              ('p', 12449),
              ('d', 33447),
              ('a', 63326),
              ('y', 22166),
              ('u', 29897),
              ('h', 54378),
              (',', 19846),
              ('m', 25083),
              ('k', 8672),
              ('.', 7885),
              ('l', 37215),
              ('v', 8591),
              ('?', 2462),
              ("'", 6187),
              ('g', 15755),
              (';', 3628),
              ('!', 2172),
              ('j', 948),
              ('-', 1897),
              ('q', 840),
              ('x', 641),


In [56]:
tokenizer.document_count

1115394

In [57]:
text_encoded = np.array(tokenizer.texts_to_sequences(text))
text_encoded = text_encoded.reshape(-1)

In [58]:
text_encoded

array([20,  6,  9, ..., 21, 27, 11])

In [59]:
text_encoded -= 1

In [89]:
window_size = 101
buffer_size = 20000
batch_size = 64


train_size = 80*tokenizer.document_count // 100
dataset_window = tf.data.Dataset.from_tensor_slices(text_encoded[:train_size])

dataset_window = dataset_window.window(size=window_size, stride=1, drop_remainder=True)

dataset_window = dataset_window.flat_map(lambda X : X.batch(window_size))
dataset_window = dataset_window.shuffle(buffer_size=buffer_size).batch(batch_size)
dataset_window = dataset_window.map(lambda X : (X[:, :-1], X[:, 1:]))
dataset = dataset_window.map(lambda X, y : (tf.one_hot(X, n_characters), y))

dataset = dataset.prefetch(tf.data.AUTOTUNE)

In [90]:
for X, y in dataset.take(1):
  print(X.shape, y.shape)

(64, 100, 39) (64, 100)


64 as batch, 100 as window size, and 39 as cardinality of the dataset.

In [70]:
epochs = 20
model = keras.Sequential([
                          keras.layers.GRU(256, dropout=0.2, input_shape=[None, n_characters], return_sequences=True),
                          keras.layers.GRU(512, dropout=0.2, return_sequences=True),
                          keras.layers.GRU(256, dropout=0.2, return_sequences=True),
                          keras.layers.TimeDistributed(keras.layers.Dense(n_characters)),
                          keras.layers.Activation("softmax")
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
model.fit(dataset, epochs=epochs)

Epoch 1/20
139/139 [==============================] - 11s 40ms/step - loss: 2.6880
Epoch 2/20
139/139 [==============================] - 7s 40ms/step - loss: 2.2170
Epoch 3/20
139/139 [==============================] - 7s 40ms/step - loss: 2.0344
Epoch 4/20
139/139 [==============================] - 7s 41ms/step - loss: 1.9264
Epoch 5/20
139/139 [==============================] - 7s 40ms/step - loss: 1.8644
Epoch 6/20
139/139 [==============================] - 7s 41ms/step - loss: 1.8234
Epoch 7/20
139/139 [==============================] - 7s 41ms/step - loss: 1.7895
Epoch 8/20
139/139 [==============================] - 7s 40ms/step - loss: 1.7664
Epoch 9/20
139/139 [==============================] - 7s 41ms/step - loss: 1.7450
Epoch 10/20
139/139 [==============================] - 7s 40ms/step - loss: 1.7342
Epoch 11/20
139/139 [==============================] - 7s 41ms/step - loss: 1.7166
Epoch 12/20
139/139 [==============================] - 7s 41ms/step - loss: 1.7044
Epoch 13/20


In [66]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15448224018725612992
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14509932544
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13031767698477494428
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


In [87]:
def preprocess(input):
  return tf.one_hot(np.array(tokenizer.texts_to_sequences(input))-1, n_characters)

def next_character(input, t=1):
  preprocessing_text = preprocess([input])
  X_new = model.predict(preprocessing_text)
  X_new = X_new[0, -1:, :]
  prob = tf.math.log(X_new) / t
  character = tf.random.categorical(prob, num_samples=1)+1
  return tokenizer.sequences_to_texts(character.numpy())[0]

string = "Hello, my na"
string += next_character(string, t=1)
string += next_character(string, t=1)
print(string)

Hello, my name


In [88]:
def generate_text(n_characters = 100, text = "a", t = 1):
  for _ in range(n_characters):
    text += next_character(text, t=t)
  return text


print(generate_text(n_characters = 50, text = "t", t = 1))

that come the mile weach of the news.

second murde
